XGBoost is a regression tree based ensemble learning technique. It has several noticeable improvements over classical gradient boosting. The major difference between gradient boosting and XGBoost is the former can be viewed in term of gradient descent while XGBoost belongs to Newton Boosting, using the second order derivative of the loss function to make a better approximation about the direction of maximum decrease. This allows the model to converge at a faster rate.  

In this work, XGBoost algorithm was coded from scratch using the exact greedy search approach for the tree method. This approach was selected as it could provide the most accurate result.  

For an optimal modal, tuning the hyperparameter is required.   
To control overfitting :
1. Directly control model complexity:
    max_depth, min_child_weight and gamma.
2. To add randomness to make training robust to noise:
    -subsample and colsample_bytree (random features used to train each tree.)
    -reduce learning rate and n_estimators (boosting rounds)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt 
from tqdm import tqdm 

In [2]:
df = pd.read_csv('E:\XGBoost\dataset\sisfall_processed_100Hz.csv')

In [3]:
df.head()

,Mean_Ax,Std_Ax,Var_Ax,Range_Ax,Mean_Ay,Std_Ay,Var_Ay,Range_Ay,Mean_Az,Std_Az,...,Range_SVM,Mean_SVM_Horizontal,Std_SVM_Horizontal,Var_SVM_Horizontal,Range_SVM_Horizontal,Mean_Angle_z_xy,Std_Angle_z_xy,Var_Angle_z_xy,Range_Angle_z_xy,label
0,0.024154,0.017176,0.000295,0.121094,-0.997331,0.020355,0.000414,0.156250,0.112057,0.069145,...,0.145789,0.132886,0.023295,0.000543,0.230243,1.458043,0.067067,0.004498,0.400908,0
1,0.009993,0.015422,0.000238,0.105469,-0.974901,0.052550,0.002762,0.183594,-0.305284,0.185734,...,0.152861,0.305947,0.185550,0.034429,0.531262,1.871868,0.183874,0.033810,0.520666,0
2,0.008552,0.010665,0.000114,0.078125,-1.019453,0.008987,0.000081,0.074219,-0.061072,0.019361,...,0.075036,0.062571,0.019394,0.000376,0.116146,1.630598,0.018832,0.000355,0.114490,0
3,0.028919,0.022021,0.000485,0.136719,-0.981068,0.052425,0.002748,0.281250,-0.163958,0.299944,...,0.256702,0.274750,0.206211,0.042523,0.578840,1.727167,0.291886,0.085198,0.741900,0
4,0.011057,0.013112,0.000172,0.089844,-1.012497,0.017493,0.000306,0.132812,-0.045383,0.055758,...,0.096304,0.050046,0.054379,0.002957,0.273577,1.616008,0.056829,0.003230,0.295768,0


In [4]:
df.shape

(4500, 25)

In [5]:
df.isnull().sum() #to ensure no missing value 

Mean_Ax                 0
Std_Ax                  0
Var_Ax                  0
Range_Ax                0
Mean_Ay                 0
Std_Ay                  0
Var_Ay                  0
Range_Ay                0
Mean_Az                 0
Std_Az                  0
Var_Az                  0
Range_Az                0
Mean_SVM                0
Std_SVM                 0
Var_SVM                 0
Range_SVM               0
Mean_SVM_Horizontal     0
Std_SVM_Horizontal      0
Var_SVM_Horizontal      0
Range_SVM_Horizontal    0
Mean_Angle_z_xy         0
Std_Angle_z_xy          0
Var_Angle_z_xy          0
Range_Angle_z_xy        0
label                   0
dtype: int64

In [6]:
df['label'].value_counts()

0    2702
1    1798
Name: label, dtype: int64

In [7]:
X = df.drop('label', axis=1)
y = df['label']

Define all the classes, functions and parameters used for XGBoost Classifier.

To construct a regression tree, class node was defined. 
Trees are recursive data structures, so the node class represents a decision point in the model. That decision point will divide the data into two sets (left and right).

The principle of the regression tree is based on https://curiousily.com/posts/build-a-decision-tree-from-scratch-in-python/

In [8]:
class Node:
    def __init__(self, X, gradient, hessian, idxs, colsample_features, min_leaf = 5, min_child_weight = 1 ,depth = 6, lambda_ = 1, gamma = 1):
      
        self.X = X #Training data in dataframe
        self.gradient = gradient #negative first derivative of loss function
        self.hessian = hessian #second derivative of loss function
        self.idxs = idxs #stores indexes of subset of the data that the particular node is working with
        self.depth = depth #maximum depth of a tree
        self.min_leaf = min_leaf #min number of samples for a node to be considered a node
        self.lambda_ = lambda_ #regularization term
        self.gamma  = gamma #prevent overfitting 
        self.min_child_weight = min_child_weight #controls model complexity (hessian)
        self.row_count = len(idxs) #no of samples of each node is working with
        self.colsample_features = colsample_features #every tree is trained with different features
        #self.colsample_by_tree = colsample_by_tree #random features used to train each tree.
        
        self.val = -np.sum(gradient[self.idxs])/(np.sum(hessian[self.idxs]) + self.lambda_) #output value for the leaf node         
        self.score = float("-inf") #internal gain score is initially set to infinity -> only leaf nodes will have infinity score.
        self.find_varsplit()
        
        
    def find_varsplit(self):

        for c in self.colsample_features: #scans through each colume (features)
            self.find_better_split(c) #find better feature to split on using the exact greed search approach
            
        if self.is_leaf: #if not such feature [at leaf node], do nothing (no splits)
            return #return None
        
        X = self.X.values[self.idxs , self.var_idx] #split based on the "best" feature
        
        #create data for left and right nodes (return the indices)
        lhs = np.nonzero(X <= self.split)[0]
        rhs = np.nonzero(X > self.split)[0]
        
        #create left and right node using the subset of the data - building recursively
        #Depth is only parameter to change as a new layer is added to tree structure.
        self.lhs = Node(X = self.X, gradient = self.gradient, hessian = self.hessian, idxs = self.idxs[lhs], min_leaf = self.min_leaf, depth = self.depth-1, lambda_ = self.lambda_ , gamma = self.gamma, min_child_weight = self.min_child_weight, colsample_features = self.colsample_features)
        self.rhs = Node(X = self.X, gradient = self.gradient, hessian = self.hessian, idxs = self.idxs[rhs], min_leaf = self.min_leaf, depth = self.depth-1, lambda_ = self.lambda_ , gamma = self.gamma, min_child_weight = self.min_child_weight, colsample_features = self.colsample_features)
        
    def find_better_split(self, var_idx):
        
        X = self.X.values[self.idxs, var_idx] #return a 1d list of all the sample value for a given feature
        
        #trying to split on each sample (datapoint) and let the best split wins with the highest score
        for r in range(self.row_count):
            lhs = X <= X[r] #return True/False in list
            rhs = X > X[r]
            
            lhs_indices = np.nonzero(X <= X[r])[0] # index lhs of split
            rhs_indices = np.nonzero(X > X[r])[0] # index rhs of split
            
            #continue until the stopping criteria is met
            if(rhs.sum() < self.min_leaf or lhs.sum() < self.min_leaf 
               or self.hessian[lhs_indices].sum() < self.min_child_weight
               or self.hessian[rhs_indices].sum() < self.min_child_weight): continue

            #In XGBoost, internal gain is used to find the optimal split value uses both the gradient and hessian
            curr_score = self.gain(lhs, rhs) 
            
            
            if curr_score > self.score: #globally updates the better score and split point 
                self.var_idx = var_idx #best feature - index of the split variable
                self.score = curr_score #gain score
                self.split = X[r]  #best "threshold" - value of the split

    def gain(self, lhs, rhs): #calculate the gain at a particular split point
        
        gradient = self.gradient[self.idxs] #gradient value for every sample 
        hessian  = self.hessian[self.idxs] #hessian value for every sample
        
        lhs_gradient = gradient[lhs].sum()
        lhs_hessian  = hessian[lhs].sum()
        
        rhs_gradient = gradient[rhs].sum()
        rhs_hessian  = hessian[rhs].sum()
        
        gain = 0.5 *((lhs_gradient**2/(lhs_hessian + self.lambda_)) + (rhs_gradient**2/(rhs_hessian + self.lambda_)) - ((lhs_gradient + rhs_gradient)**2/(lhs_hessian + rhs_hessian + self.lambda_))) - self.gamma
        return(gain)
                
    @property
    def is_leaf(self):
        #score is initialize as infinity and only leaf node has infinity score (gain)
        return self.score == float('-inf') or self.depth <= 0                 

    def predict(self, X):
        return np.array([self.predict_row(x) for x in X]) #the prediction is stored in array. 
    
    def predict_row(self, x):
    #check if need to go left or right node based on the split value and feature found
        if self.is_leaf: #if leaf node, compute the output value
            return(self.val)

        node = self.lhs if x[self.var_idx] <= self.split else self.rhs
        return node.predict_row(x)

In [9]:
#Build the regression tree 
class XGBoostTree:
    def fit(self,X,gradient,hessian,colsample_features,min_leaf=5,min_child_weight=1,depth=6,lambda_=1,gamma=1):
        self.XGBoosttree = Node(X,gradient,hessian,np.array(np.arange(len(X))),colsample_features,min_leaf,min_child_weight,depth,lambda_, gamma)
        return self
       
    def predict(self, X):
        return self.XGBoosttree.predict(X.values)

XGBoost Classification (XGBoost Tree in loop)

In [10]:
def sigmoid(x): #function to convert from log odds to probability format
    return 1/(1+np.exp(-x))

#gradient is the negative derivative of log loss
#prediciton in probability, y_label is the true label of the sample
def cal_gradient(prediction,y_label):
    p = sigmoid(prediction)
    return (p-y_label)
    
#hessian is the second derivative of the log loss
def cal_hessian(prediction):
    p = sigmoid(prediction)
    return (p*(1-p))

#XGBoost will subsample the ratio of the training instances to train a single tree
#randomly select the mentioned ratio of training data prior to grow a tree 
#to reduce the variance and the dependence on certain values in the training sample
def subsample(X_train, y_train, subsample_ratio):
    n_samples = len(X_train)
    n_samples = np.arange(n_samples)
    #idxs = random.sample(n_samples, np.round(subsample_ratio*n_samples)) #without replacement
    idxs = np.random.choice(n_samples, round(subsample_ratio*len(n_samples)), replace=False)
    return X_train.iloc[idxs], y_train.iloc[idxs]

#reduce the variance of the models by training on different parts of the data.
#each tree is trained with random features 
def colsample_bytree(X_train, colsample_ratio):
    n_features = X.shape[1]
    feature_idxs = np.random.choice(n_features, round(colsample_ratio*n_features), replace=True)
    return feature_idxs

Evaluation metrics to evaluate the performance of the model

In [11]:
#Compute TP, TN, FN, FP
'''
True_positive: Actual = 1, Predicted = 1
False_positive: Acutal = 0, Predicted = 1
True_negative: Acutal = 0, Predicted = 0
False_negative: Acutal = 1, Predicted = 0
'''
def compute_TP_TN_FP_FN(y_actual, y_predict):
    tp = sum((y_actual == 1) & (y_predict == 1))
    fp = sum((y_actual == 0) & (y_predict == 1))
    tn = sum((y_actual == 0) & (y_predict == 0))
    fn = sum((y_actual == 1) & (y_predict == 0))
    
    return tp, fp, tn, fn

def accuracy(TP, TN, FP, FN):
    acc = (TP + TN) / (TP + FP + TN + FN)
    return acc

def sensitivity (TP, FN): #recall
    sensi = TP / (TP + FN)
    return sensi

def specificity (TN, FP):
    speci = TN / (TN + FP)
    return speci

def precision(TP, FP):
    preci = TP / (TP + FP)
    return preci

def F1_Score(precision, sensitivity): 
#maximum score of 1 (perfect precision and recall) and a minimum of 0 
    F1 = (2 * precision * sensitivity) / (precision + sensitivity)
    return F1

def compute_FPR(sensitivity): #False positive rate 
    FPR = 1 - sensitivity
    return FPR

Initialization of the hyperparameters

In [12]:
#Initialization
learning_rate = 0.3
lambda_ = 1
gamma = 1
depth = 10
min_child_weight = 1
subsample_ratio = 1
colsample_ratio = 0.8
boosting_rounds = 10

10 Fold cross validation to train the model - prevent overfitting. Each fold: 450 samples

In [13]:
k = StratifiedKFold(n_splits=10, shuffle = True)
 
TP_ = []
FP_ = []
TN_ = []
FN_ = []
acc_ = []
sensi_ = []
speci_ = []
preci_ = []
F1_ = []
FPR_ = []

for train_index, test_index in k.split(X,y):
    #train = df.iloc[train_index]
    #test = df.iloc[test_index] 
    #split the dataset into training and test set
    #X_train, y_train = train.iloc[:,:-1], train.iloc[:,-1]
    #X_test, y_test = test.iloc[:,:-1], test.iloc[:,-1]
    
    n_estimators = []
    #print("Ori_n_estimators.shape", np.shape(n_estimators))
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
               
    #train the classifier with X_train
    base_prediction = np.full((round(subsample_ratio*X_train.shape[0]),1),1).flatten().astype('float64') 
    
    #for each regression tree    
    #boosting_rounds refer to the n_estimators in XGBoost algorithm. 
    #numbers of the regression tree to be built
    for i in range (boosting_rounds):   
        
        #randomly select samples to train the model 
        random_X, random_y = subsample(X_train, y_train, subsample_ratio)
            
        #randomly select features to train the model
        colsample_features = colsample_bytree(X_train, colsample_ratio)

        #use the initial prediction to calculate gradient and hessian
        gradient = cal_gradient(base_prediction, random_y.values) #return 1d array
        hessian = cal_hessian(base_prediction) #return 1d array
            
        #build the tree
        tree = XGBoostTree()
        boosting_tree = tree.fit(random_X,gradient,hessian,colsample_features,min_leaf=5,min_child_weight=1,depth=8,lambda_=1,gamma=1)
            
        #predict the boosting tree and update the base prediction 
        base_prediction += learning_rate * boosting_tree.predict(random_X)
        n_estimators.append(boosting_tree) #save the regreesion tree to the list        
        
    #Prediction for X_test
    
    #create an empty array to store the latest prediction for X_test
    prediction = np.zeros(X_test.shape[0]) #initial prediction of 0 for all the testing data

    #initial prediction for testing data, which is 1
    initial_pred = np.full((X_test.shape[0],1),1).flatten().astype('float64')
    
    #update the prediction for every new created regression tree 
    #for every tree in all the boosting trees created
    for estimator in n_estimators:
        #new_prediction is in log_odds format for X_test
        prediction += learning_rate * estimator.predict(X_test) #predict the boosting tree  
        
    #final prediction in log odds format
    new_prediction = initial_pred + prediction
    #convert from log_odds into probability
    new_prediction_prob = sigmoid(new_prediction)
    
    #set a threshold
    #if the new_prediction > "value", classify into 1 - high risk and vice versa
    #prediction for X_test
    y_pred = np.where(new_prediction_prob > np.mean(new_prediction_prob), 1, 0)
    print("pred", np.shape(y_pred))

                
    #evaluation for test set btw y_test and y_pred
    TP, FP, TN, FN = compute_TP_TN_FP_FN(y_test, y_pred)
    TP_.append(TP), FP_.append(FP), TN_.append(TN), FN_.append(FN)
    acc = accuracy(TP, TN, FP, FN)
    acc_.append(acc)
    sensi = sensitivity (TP, FN)
    sensi_.append(sensi)
    speci = specificity (TN, FP)
    speci_.append(speci)
    preci = precision(TP, FP)
    preci_.append(preci)
    F1 = F1_Score(preci, sensi)
    F1_.append(F1)
    FPR = compute_FPR(sensi)
    FPR_.append(FPR)

Ori_n_estimators.shape (0,)
Size of n_estimators (10,)
pred (450,)
Ori_n_estimators.shape (0,)
Size of n_estimators (10,)
pred (450,)
Ori_n_estimators.shape (0,)
Size of n_estimators (10,)
pred (450,)
Ori_n_estimators.shape (0,)
Size of n_estimators (10,)
pred (450,)
Ori_n_estimators.shape (0,)
Size of n_estimators (10,)
pred (450,)
Ori_n_estimators.shape (0,)
Size of n_estimators (10,)
pred (450,)
Ori_n_estimators.shape (0,)
Size of n_estimators (10,)
pred (450,)
Ori_n_estimators.shape (0,)
Size of n_estimators (10,)
pred (450,)
Ori_n_estimators.shape (0,)
Size of n_estimators (10,)
pred (450,)
Ori_n_estimators.shape (0,)
Size of n_estimators (10,)
pred (450,)


In [14]:
print("TP: ", TP_)
print("FP: ", FP_)
print("TN: ", TN_)
print("FN: ", FN_)
print("Accuracy: ", acc_)
print("Sensitivity: ", sensi_)
print("Specificity: ", speci_)
print("Precision: ", preci_)
print("F1_Score: ", F1_)
print("FPR: ", FPR_)

TP:  [179, 175, 176, 173, 172, 178, 178, 174, 177, 175]
FP:  [9, 2, 3, 5, 6, 11, 9, 4, 7, 7]
TN:  [262, 269, 267, 265, 264, 259, 261, 266, 263, 263]
FN:  [0, 4, 4, 7, 8, 2, 2, 6, 3, 5]
Accuracy:  [0.98, 0.9866666666666667, 0.9844444444444445, 0.9733333333333334, 0.9688888888888889, 0.9711111111111111, 0.9755555555555555, 0.9777777777777777, 0.9777777777777777, 0.9733333333333334]
Sensitivity:  [1.0, 0.9776536312849162, 0.9777777777777777, 0.9611111111111111, 0.9555555555555556, 0.9888888888888889, 0.9888888888888889, 0.9666666666666667, 0.9833333333333333, 0.9722222222222222]
Specificity:  [0.966789667896679, 0.992619926199262, 0.9888888888888889, 0.9814814814814815, 0.9777777777777777, 0.9592592592592593, 0.9666666666666667, 0.9851851851851852, 0.9740740740740741, 0.9740740740740741]
Precision:  [0.9521276595744681, 0.9887005649717514, 0.9832402234636871, 0.9719101123595506, 0.9662921348314607, 0.9417989417989417, 0.9518716577540107, 0.9775280898876404, 0.9619565217391305, 0.961538461

Compute the mean value for the 10-fold CV

In [15]:
TP_mean= round(np.mean(TP_))
print("TP: ", TP_mean)
FP_mean = round(np.mean(FP_))
print("FP: ", FP_mean)
TN_mean= round(np.mean(TN_))
print("TN: ", TN_mean)
FN_mean= round(np.mean(FN_))
print("FN: ", FN_mean)

TP:  176
FP:  6
TN:  264
FN:  4


In [16]:
Acc_mean = round(np.mean(acc_),2)
Acc_std = round(np.std(acc_),2)
print("Accuracy: ", Acc_mean, "+-", Acc_std)

Sensi_mean = round(np.mean(sensi_),2)
Sensi_std= round(np.std(sensi_),2)
print("Sensitivity: ", Sensi_mean, "+-", Sensi_std)

Speci_mean = round(np.mean(speci_),2)
Speci_std = round(np.mean(speci_),2)
print("Specificity: ", Speci_mean, "+-", Speci_std)

Preci_mean = round(np.mean(preci_),2)
Preci_std = round(np.std(preci_),2)
print("Precision: ", Preci_mean, "+-", Preci_std)

F1_mean = round(np.mean(F1_),2)
F1_std = round(np.std(F1_),2)
print("F1: ", F1_mean, "+-", F1_std)

FPR_mean = round(np.mean(FPR_),2)
FPR_std = round(np.std(FPR_),2)
print("FPR: ", FPR_mean, "+-", FPR_std)

Accuracy:  0.98 +- 0.01
Sensitivity:  0.98 +- 0.01
Specificity:  0.98 +- 0.98
Precision:  0.97 +- 0.01
F1:  0.97 +- 0.01
FPR:  0.02 +- 0.01


Hyperparameter tuning

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1234,stratify=y)

TP_ = []
FP_ = []
TN_ = []
FN_ = []
acc_ = []
sensi_ = []
speci_ = []
preci_ = []
F1_ = []
FPR_ = []

for boosting_round in [5,10,15]:   #changing the parameter
    n_estimators = []
    print("Ori_n_estimators.shape", np.shape(n_estimators))
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
               
    #train the classifier with X_train
    base_prediction = np.full((round(subsample_ratio*X_train.shape[0]),1),1).flatten().astype('float64') 
    
    #for each regression tree    
    #boosting_rounds refer to the n_estimators in XGBoost algorithm. 
    #numbers of the regression tree to be built
    for i in range (boosting_rounds):   
        
        #randomly select samples to train the model 
        random_X, random_y = subsample(X_train, y_train, subsample_ratio)
            
        #randomly select features to train the model
        colsample_features = colsample_bytree(X_train, colsample_ratio)

        #use the initial prediction to calculate gradient and hessian
        gradient = cal_gradient(base_prediction, random_y.values) #return 1d array
        hessian = cal_hessian(base_prediction) #return 1d array
            
        #build the tree
        tree = XGBoostTree()
        boosting_tree = tree.fit(random_X,gradient,hessian,colsample_features,min_leaf=5,min_child_weight=1,depth=8,lambda_=1,gamma=1)
            
        #predict the boosting tree and update the base prediction 
        base_prediction += learning_rate * boosting_tree.predict(random_X)
        n_estimators.append(boosting_tree) #save the regreesion tree to the list        
        
    #prediction for X_test
    print("Size of n_estimators", np.shape(n_estimators))
    #create an empty array to store the latest prediction for X_test
    prediction = np.zeros(X_test.shape[0]) #initial prediction of 0 for all the testing data

    #initial prediction for testing data, which is 1
    initial_pred = np.full((X_test.shape[0],1),1).flatten().astype('float64')
    
    #update the prediction for every new created regression tree 
    #for every tree in all the boosting trees created
    for estimator in n_estimators:
        #new_prediction is in log_odds format for X_test
        prediction += learning_rate * estimator.predict(X_test) #predict the boosting tree  
        
    #final prediction in log odds format
    new_prediction = initial_pred + prediction
    #convert from log_odds into probability
    new_prediction_prob = sigmoid(new_prediction)
    
    #set a threshold
    #if the new_prediction > "value", classify into 1 - high risk and vice versa
    #prediction for X_test
    y_pred = np.where(new_prediction_prob > np.mean(new_prediction_prob), 1, 0)
    print("pred", np.shape(y_pred))

                
    #evaluation for test set btw y_test and y_pred
    TP, FP, TN, FN = compute_TP_TN_FP_FN(y_test, y_pred)
    TP_.append(TP), FP_.append(FP), TN_.append(TN), FN_.append(FN)
    acc = accuracy(TP, TN, FP, FN)
    acc_.append(acc)
    sensi = sensitivity (TP, FN)
    sensi_.append(sensi)
    speci = specificity (TN, FP)
    speci_.append(speci)
    preci = precision(TP, FP)
    preci_.append(preci)
    F1 = F1_Score(preci, sensi)
    F1_.append(F1)
    FPR = compute_FPR(sensi)
    FPR_.append(FPR)